Import the necessary packages:

In [1]:
import pandas as pd
import numpy as np
import yaml
from pprint import pprint as pprint
import pickle
from pandas.io.json import json_normalize

### Movie Credits: Cast and Crew data:
Let's start by looking at the movie cast and crew data.

Load the data as a __Pandas__ DataFrame object and take a look at the first 5 rows of the data:

In [2]:
creditsDF = pd.read_csv("C:\\Users\\vosilr2\\Documents\\NAMV_MLcontest\\data\\credits.csv")
creditsDF.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [3]:
print(f'There are {creditsDF.shape[0]} observations in the credits dataset.')

There are 45476 observations in the credits dataset.


##### Missing values:
Dealing with missing values is always important. Removing the rows with missing data might not always be the best idea. The fact that data is missing for a certain observation of a certain variable might in itself be useful data. So, that's something to have in mind.

In this dataset, there are a some rows that have no data in the `cast` or in `crew` columns. We can remove the rows where both of the columns have missing data and leave the one's where only one of them has missing observations. 

There are $421$ (less than 1% of the data) such rows where we have information on neither the movie cast nor the crew:

In [4]:
len(creditsDF[(creditsDF.cast == "[]") & (creditsDF.crew == "[]")])

421

In [5]:
creditsDF = creditsDF[(creditsDF.cast != "[]") | (creditsDF.crew != "[]")]
print(f'After removing the above rows, there are now {len(creditsDF)} obserations in the credits dataset.')

After removing the above rows, there are now 45055 obserations in the credits dataset.


##### A closer look:

Let's examine one of the observations in the `cast` column:

In [6]:
creditsDF.cast[0]

"[{'cast_id': 14, 'character': 'Woody (voice)', 'credit_id': '52fe4284c3a36847f8024f95', 'gender': 2, 'id': 31, 'name': 'Tom Hanks', 'order': 0, 'profile_path': '/pQFoyx7rp09CJTAb932F2g8Nlho.jpg'}, {'cast_id': 15, 'character': 'Buzz Lightyear (voice)', 'credit_id': '52fe4284c3a36847f8024f99', 'gender': 2, 'id': 12898, 'name': 'Tim Allen', 'order': 1, 'profile_path': '/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg'}, {'cast_id': 16, 'character': 'Mr. Potato Head (voice)', 'credit_id': '52fe4284c3a36847f8024f9d', 'gender': 2, 'id': 7167, 'name': 'Don Rickles', 'order': 2, 'profile_path': '/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg'}, {'cast_id': 17, 'character': 'Slinky Dog (voice)', 'credit_id': '52fe4284c3a36847f8024fa1', 'gender': 2, 'id': 12899, 'name': 'Jim Varney', 'order': 3, 'profile_path': '/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg'}, {'cast_id': 18, 'character': 'Rex (voice)', 'credit_id': '52fe4284c3a36847f8024fa5', 'gender': 2, 'id': 12900, 'name': 'Wallace Shawn', 'order': 4, 'profile_path': '/oGE6JqPP2xH4t

It seems that the `cast` and `crew` columns contain JSON type data in string format. These strings could be converted into dictionaries and then expanded afterwards. There are several Python packages that can help us with that. `yaml` is one of them. Below we apply the `yaml.load()` method on the two columns, which returns two columns with a list of dictionaries in them:

In [7]:
creditsDF[["cast", "crew"]] = creditsDF[["cast", "crew"]].applymap(yaml.load)

In [8]:
creditsDF.cast[0]

[{'cast_id': 14,
  'character': 'Woody (voice)',
  'credit_id': '52fe4284c3a36847f8024f95',
  'gender': 2,
  'id': 31,
  'name': 'Tom Hanks',
  'order': 0,
  'profile_path': '/pQFoyx7rp09CJTAb932F2g8Nlho.jpg'},
 {'cast_id': 15,
  'character': 'Buzz Lightyear (voice)',
  'credit_id': '52fe4284c3a36847f8024f99',
  'gender': 2,
  'id': 12898,
  'name': 'Tim Allen',
  'order': 1,
  'profile_path': '/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg'},
 {'cast_id': 16,
  'character': 'Mr. Potato Head (voice)',
  'credit_id': '52fe4284c3a36847f8024f9d',
  'gender': 2,
  'id': 7167,
  'name': 'Don Rickles',
  'order': 2,
  'profile_path': '/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg'},
 {'cast_id': 17,
  'character': 'Slinky Dog (voice)',
  'credit_id': '52fe4284c3a36847f8024fa1',
  'gender': 2,
  'id': 12899,
  'name': 'Jim Varney',
  'order': 3,
  'profile_path': '/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg'},
 {'cast_id': 18,
  'character': 'Rex (voice)',
  'credit_id': '52fe4284c3a36847f8024fa5',
  'gender': 2,
  'id': 12900,
 

In [9]:
creditsDF.rename(index=str,columns={"id": "movieId"}, inplace=True)

Now that all observations have been converted into lists of dictionaries, we can use `Panda`'s `json_normalize` function to expand or flatten the dataset. We'll create two separate datasets for cast and crew and then merge them. However, first we need to make sure the movie ID is also present in both of these. For that we can make a function that will create a dictionary of dictionaries containing the movie id and the cast dictionary for each observation:

In [10]:
def makeDicts(row):
    '''
    input: a list of two items - a dictionary of cast data and a digit representing movie ID
    output: a dictionary comprised of the two inputs.
    '''
    if len(row[0]) != 0:
        x = {row.index[1] : row[1],
             row.index[0] : row[0][:len(row[0])]}
    else:
        x = np.NaN
    return x

#### Cast data:

Apply the `makeDicts` function for each row of the `cast` and `id` columns and create a temporary dataframe:

In [11]:
newDF = creditsDF.iloc[:,[0, 2]].apply(makeDicts, axis=1)

drop the observations with `NaN`:

In [12]:
newDF.dropna(inplace=True)
newDF

0        {'movieId': 862, 'cast': [{'cast_id': 14, 'cha...
1        {'movieId': 8844, 'cast': [{'cast_id': 1, 'cha...
2        {'movieId': 15602, 'cast': [{'cast_id': 2, 'ch...
3        {'movieId': 31357, 'cast': [{'cast_id': 1, 'ch...
4        {'movieId': 11862, 'cast': [{'cast_id': 1, 'ch...
5        {'movieId': 949, 'cast': [{'cast_id': 25, 'cha...
6        {'movieId': 11860, 'cast': [{'cast_id': 1, 'ch...
7        {'movieId': 45325, 'cast': [{'cast_id': 2, 'ch...
8        {'movieId': 9091, 'cast': [{'cast_id': 1, 'cha...
9        {'movieId': 710, 'cast': [{'cast_id': 1, 'char...
10       {'movieId': 9087, 'cast': [{'cast_id': 1, 'cha...
11       {'movieId': 12110, 'cast': [{'cast_id': 9, 'ch...
12       {'movieId': 21032, 'cast': [{'cast_id': 1, 'ch...
13       {'movieId': 10858, 'cast': [{'cast_id': 1, 'ch...
14       {'movieId': 1408, 'cast': [{'cast_id': 1, 'cha...
15       {'movieId': 524, 'cast': [{'cast_id': 4, 'char...
16       {'movieId': 4584, 'cast': [{'cast_id': 6, 'cha.

... and finally create the `cast` dataframe:

In [13]:
castDF = json_normalize(newDF, "cast", "movieId")

Let's print it out:

In [14]:
castDF

,cast_id,character,credit_id,gender,id,name,order,profile_path,movieId
0,14,Woody (voice),52fe4284c3a36847f8024f95,2,31,Tom Hanks,0,/pQFoyx7rp09CJTAb932F2g8Nlho.jpg,862
1,15,Buzz Lightyear (voice),52fe4284c3a36847f8024f99,2,12898,Tim Allen,1,/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg,862
2,16,Mr. Potato Head (voice),52fe4284c3a36847f8024f9d,2,7167,Don Rickles,2,/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg,862
3,17,Slinky Dog (voice),52fe4284c3a36847f8024fa1,2,12899,Jim Varney,3,/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg,862
4,18,Rex (voice),52fe4284c3a36847f8024fa5,2,12900,Wallace Shawn,4,/oGE6JqPP2xH4tNORKNqxbNPYi7u.jpg,862
5,19,Hamm (voice),52fe4284c3a36847f8024fa9,2,7907,John Ratzenberger,5,/yGechiKWL6TJDfVE2KPSJYqdMsY.jpg,862
6,20,Bo Peep (voice),52fe4284c3a36847f8024fad,1,8873,Annie Potts,6,/eryXT84RL41jHSJcMy4kS3u9y6w.jpg,862
7,26,Andy (voice),52fe4284c3a36847f8024fc1,0,1116442,John Morris,7,/vYGyvK4LzeaUCoNSHtsuqJUY15M.jpg,862
8,22,Sid (voice),52fe4284c3a36847f8024fb1,2,12901,Erik von Detten,8,/twnF1ZaJ1FUNUuo6xLXwcxjayBE.jpg,862
9,23,Mrs. Davis (voice),52fe4284c3a36847f8024fb5,1,12133,Laurie Metcalf,9,/unMMIT60eoBM2sN2nyR7EZ2BvvD.jpg,862


###### Number of cast members per movie:

In [50]:
l = castDF.movieId[castDF.order >= 100].unique()
print('There are', len(l), 'movies with 100 or more cast members listed in the dataset.')

There are 94 movies with 100 or more cast members listed in the dataset.


In [48]:
l = castDF.movieId[castDF.order <= 10].unique()
print('There are', len(l), 'movies with 10 or less cast members listed in the dataset.')

There are 42998 movies with 10 or less cast members listed in the dataset.


###### Movie roles per actress/actor:

Let's see who's the most prolific actor/actress in our database:

In [15]:
valueCounts = castDF.name.value_counts()

In [16]:
print(valueCounts[valueCounts == valueCounts.max()])

Bess Flowers    241
Name: name, dtype: int64


That seems about right. According to [this HollywoodReporter page](https://www.hollywoodreporter.com/race/20-feet-movie-stardom-overlooked-682162) Bess Flowers is known as:
> "_the Queen of the Hollywood Extras_"

Christopher Lee is in second place with 148 appearences. Below is a list of actors and actresses that have appeared in 100 or more movies, with Canada's own Donald Sutherland at number 7.

In [17]:
print(valueCounts[valueCounts >= 100])

Bess Flowers         241
Christopher Lee      148
John Wayne           125
Samuel L. Jackson    123
Michael Caine        110
Gérard Depardieu     110
John Carradine       109
Donald Sutherland    109
Jackie Chan          108
Frank Welker         107
Robert De Niro       104
Danny Trejo          102
Steve Buscemi        101
Name: name, dtype: int64


In [18]:
print(f'There are {len(valueCounts)} actors/actresses in the dataset.')
print(f'{np.round(len(valueCounts[valueCounts==1])/len(valueCounts),3)*100}% ({len(valueCounts[valueCounts==1])} actors/actresses) of them appeared only in one movie.')

There are 202747 actors/actresses in the dataset.
63.9% (129508 actors/actresses) of them appeared only in one movie.


 Bess Flowers (#1) and Susan Sarandon(#20) are the only women on the top 20 list of movie appearences:

In [19]:
print(valueCounts[:20])

Bess Flowers          241
Christopher Lee       148
John Wayne            125
Samuel L. Jackson     123
Michael Caine         110
Gérard Depardieu      110
John Carradine        109
Donald Sutherland     109
Jackie Chan           108
Frank Welker          107
Robert De Niro        104
Danny Trejo           102
Steve Buscemi         101
Irving Bacon           98
John Goodman           97
Danny Glover           96
Christopher Walken     96
Dennis Hopper          94
Susan Sarandon         93
John Hurt              93
Name: name, dtype: int64


The data  in the gender columns looks inconsistent. There are three categories: 0, 1 and 2.

In [20]:
print(castDF.gender.value_counts())

2    226713
0    223964
1    111797
Name: gender, dtype: int64


Below is a slice of the dataset for gender category 0. We can see that, just by guessing the gender using the names, the data is mixed.

In [21]:
castDF[castDF.gender==0][["character", "name"]]

,character,name
7,Andy (voice),John Morris
16,Peter Shepherd,Bradley Pierce
21,Alan Parrish (young),Adam Hann-Byrd
26,Caleb,Cyrus Thiedeke
27,Billy Jessup,Gary Joseph Thorup
28,Cop,Leonard Zola
31,Martha Shepherd,Annabel Kershaw
33,Paramedic,Robyn Driscoll
35,Girl,Sarah Gilson
36,Girl,Florica Vlad


I don't think, we'll be able to use the gender column in any sensible way given that it's all mixed up.

#### Crew data:
Let's create the crew dataset:

In [22]:
tempDF = creditsDF.iloc[:,[1, 2]].apply(makeDicts, axis=1)

In [23]:
tempDF.dropna(inplace=True)
tempDF

0        {'movieId': 862, 'crew': [{'credit_id': '52fe4...
1        {'movieId': 8844, 'crew': [{'credit_id': '52fe...
2        {'movieId': 15602, 'crew': [{'credit_id': '52f...
3        {'movieId': 31357, 'crew': [{'credit_id': '52f...
4        {'movieId': 11862, 'crew': [{'credit_id': '52f...
5        {'movieId': 949, 'crew': [{'credit_id': '52fe4...
6        {'movieId': 11860, 'crew': [{'credit_id': '52f...
7        {'movieId': 45325, 'crew': [{'credit_id': '52f...
8        {'movieId': 9091, 'crew': [{'credit_id': '52fe...
9        {'movieId': 710, 'crew': [{'credit_id': '52fe4...
10       {'movieId': 9087, 'crew': [{'credit_id': '52fe...
11       {'movieId': 12110, 'crew': [{'credit_id': '52f...
12       {'movieId': 21032, 'crew': [{'credit_id': '593...
13       {'movieId': 10858, 'crew': [{'credit_id': '52f...
14       {'movieId': 1408, 'crew': [{'credit_id': '52fe...
15       {'movieId': 524, 'crew': [{'credit_id': '52fe4...
16       {'movieId': 4584, 'crew': [{'credit_id': '52fe.

In [24]:
crewDF = json_normalize(tempDF, "crew", "movieId")

In [25]:
crewDF

,credit_id,department,gender,id,job,name,profile_path,movieId
0,52fe4284c3a36847f8024f49,Directing,2,7879,Director,John Lasseter,/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg,862
1,52fe4284c3a36847f8024f4f,Writing,2,12891,Screenplay,Joss Whedon,/dTiVsuaTVTeGmvkhcyJvKp2A5kr.jpg,862
2,52fe4284c3a36847f8024f55,Writing,2,7,Screenplay,Andrew Stanton,/pvQWsu0qc8JFQhMVJkTHuexUAa1.jpg,862
3,52fe4284c3a36847f8024f5b,Writing,2,12892,Screenplay,Joel Cohen,/dAubAiZcvKFbboWlj7oXOkZnTSu.jpg,862
4,52fe4284c3a36847f8024f61,Writing,0,12893,Screenplay,Alec Sokolow,/v79vlRYi94BZUQnkkyznbGUZLjT.jpg,862
5,52fe4284c3a36847f8024f67,Production,1,12894,Producer,Bonnie Arnold,None,862
6,52fe4284c3a36847f8024f6d,Production,0,12895,Executive Producer,Ed Catmull,None,862
7,52fe4284c3a36847f8024f73,Production,2,12896,Producer,Ralph Guggenheim,None,862
8,52fe4284c3a36847f8024f79,Production,2,12897,Executive Producer,Steve Jobs,/mOMP3SwD5qWQSR0ldCIByd3guTV.jpg,862
9,52fe4284c3a36847f8024f8b,Editing,2,8,Editor,Lee Unkrich,/bdTCCXjgOV3YyaNmLGYGOxFQMOc.jpg,862


In [26]:
crewDeps = crewDF.department.value_counts()
print(f'There are {len(crewDeps)} unique Departments in the crew dataset:')
print(crewDeps)

There are 12 unique Departments in the crew dataset:
Production           94498
Writing              74831
Directing            58134
Sound                50605
Art                  40694
Camera               33539
Crew                 31605
Costume & Make-Up    30850
Editing              29831
Visual Effects       14861
Lighting              4847
Actors                  19
Name: department, dtype: int64


There is a department called "Actors" that appears 19 times. Let's check those rows out:

In [138]:
crewDF[crewDF.department == "Actors"]

,credit_id,department,gender,id,job,name,profile_path,movieId
13877,52fe4227c3a36847f800823d,Actors,0,2659,Stunt Double,Sol Gorss,None,213
37610,52fe42f1c3a36847f802e9d5,Actors,0,16500,Stunt Double,Mark De Alessandro,None,1375
43512,52fe427ac3a36847f8021c71,Actors,0,14445,Stunt Double,Leigh Walsh,None,813
56106,52fe424fc3a36847f80143c3,Actors,0,7344,Stunt Double,Martha Crawford,None,541
113958,52fe42f0c3a36847f802e7c5,Actors,0,16689,Stunt Double,Diana Wiersma,None,1373
137687,52fe46c7c3a36847f8111b93,Actors,2,1979,Voice,Kevin Spacey,/cdowETe1PgXLjo72hDb7R7tyavf.jpg,45538
166719,52fe42f0c3a36847f802e67d,Actors,0,16740,Stunt Double,Francois Grobbelaar,None,1372
166720,52fe42f0c3a36847f802e683,Actors,0,16741,Stunt Double,Mick 'Stuntie' Milligan,None,1372
213762,52fe46269251416c9104ad0d,Actors,0,119183,Stunt Double,Giorgio Antonini,None,37056
258423,52fe44a0c3a368484e02dbbf,Actors,0,132658,Voice,Fraser McGregor,None,24617


The three job categories, `Stunt Double`, `Voice` and `Actor`, do not appear anywhere else in the dataset, except the above rows.

In [154]:
smallList = crewDF[crewDF.department == "Actors"].job.unique()
print("_"*29)
for i in smallList:
    print(i + ":")
    print("-"*(len(i)+1))
    print(crewDF[crewDF.job == i].name.value_counts())
    print("_"*29)

_____________________________
Stunt Double:
-------------
Martha Crawford            1
Francois Grobbelaar        1
Giorgio Antonini           1
Leigh Walsh                1
Mick 'Stuntie' Milligan    1
Diana Wiersma              1
Sol Gorss                  1
Mark De Alessandro         1
Name: name, dtype: int64
_____________________________
Voice:
------
Kevin Spacey       1
Fraser McGregor    1
Name: name, dtype: int64
_____________________________
Actor:
------
Sven-Bertil Taube       1
Mario Marenco           1
Roberto Benigni         1
Gwynne Dyer             1
Luciano De Crescenzo    1
Diego Abatantuono       1
Andy Luotto             1
Barbara Parkins         1
Isabella Rossellini     1
Name: name, dtype: int64
_____________________________


Let's look at the crew job categories:

In [125]:
crewJobs = crewDF.job.value_counts()

In [128]:
print(f'There are {len(crewJobs)} unique crew jobs in the dataset.')

There are 456 unique crew jobs in the dataset.


In [129]:
print(crewJobs)

Director                          49048
Producer                          43555
Writer                            30431
Screenplay                        25163
Editor                            23800
Director of Photography           20659
Executive Producer                18866
Original Music Composer           15790
Casting                           12635
Art Direction                     11689
Costume Design                    11023
Production Design                 10536
Set Decoration                     7548
Music                              7340
Makeup Artist                      6402
Story                              5810
Novel                              5128
Script Supervisor                  4328
Cinematography                     4210
Camera Operator                    4128
Animation                          3834
Sound Re-Recording Mixer           3680
Associate Producer                 3595
Hairstylist                        3539
Co-Producer                        3189


In [157]:
directorCounts = crewDF[crewDF.job == "Director"].name.value_counts()

In [159]:
print(f'There are {len(directorCounts)} Directors in the dataset.')

There are 19740 Directors in the dataset.


Below are the most prolific directors:

In [161]:
print(directorCounts[:10])

John Ford             68
Michael Curtiz        65
Werner Herzog         55
Alfred Hitchcock      53
Georges Méliès        51
Jean-Luc Godard       50
Woody Allen           49
Sidney Lumet          46
Charlie Chaplin       44
William A. Wellman    43
Name: name, dtype: int64


In [175]:
print(f'{len(directorCounts[directorCounts == 1])} directors ({np.round(len(directorCounts[directorCounts == 1])/len(directorCounts),3)*100}%) have only directed one film in our dataset.')

12176 directors (61.7%) have only directed one film in our dataset.


## Movies metadata:

Let's move on to the other datasets, like the one with the movie metadata.


In [27]:
metadataDF = pd.read_csv("C:\\Users\\vosilr2\\Documents\\NAMV_MLcontest\\data\\movies_metadata.csv", low_memory=False)

In [65]:
metadataDF.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


Number of observations in the dataset:

In [29]:
len(metadataDF)

45466

The column names in the metadata dataset:

In [73]:
list(metadataDF.columns)

['adult',
 'belongs_to_collection',
 'budget',
 'genres',
 'homepage',
 'id',
 'imdb_id',
 'original_language',
 'original_title',
 'overview',
 'popularity',
 'poster_path',
 'production_companies',
 'production_countries',
 'release_date',
 'revenue',
 'runtime',
 'spoken_languages',
 'status',
 'tagline',
 'title',
 'video',
 'vote_average',
 'vote_count']

let's take a look at the unique values in the first column, `adult`:

In [85]:
print(metadataDF.adult.unique())

['False' 'True' ' - Written by Ørnås'
 ' Rune Balot goes to a casino connected to the October corporation to try to wrap up her case once and for all.'
 ' Avalanche Sharks tells the story of a bikini contest that turns into a horrifying affair when it is hit by a shark avalanche.']


That looks a bit odd, the first two values, False and True, are as expected. However, there seems to be some erroneous input in this column. Let's check the rows where the three strings in the list above are in the `adult` column:

In [89]:
metadataDF[(metadataDF.adult == ' - Written by Ørnås') | 
           (metadataDF.adult == ' Rune Balot goes to a casino connected to the October corporation to try to wrap up her case once and for all.') |
           (metadataDF.adult == ' Avalanche Sharks tells the story of a bikini contest that turns into a horrifying affair when it is hit by a shark avalanche.')]

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
19730,- Written by Ørnås,0.065736,/ff9qCepilowshEtG2GYWwzt2bs4.jpg,"[{'name': 'Carousel Productions', 'id': 11176}...","[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...",1997-08-20,0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29503,Rune Balot goes to a casino connected to the ...,1.931659,/zV8bHuSL6WXoD6FWogP9j4x80bL.jpg,"[{'name': 'Aniplex', 'id': 2883}, {'name': 'Go...","[{'iso_3166_1': 'US', 'name': 'United States o...",2012-09-29,0,68.0,"[{'iso_639_1': 'ja', 'name': '日本語'}]",Released,...,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35587,Avalanche Sharks tells the story of a bikini ...,2.185485,/zaSf5OG7V8X8gqFvly88zDdRm46.jpg,"[{'name': 'Odyssey Media', 'id': 17161}, {'nam...","[{'iso_3166_1': 'CA', 'name': 'Canada'}]",2014-01-01,0,82.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,...,22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


It seems that these three rows have data in the wrong columns, let's examine the first one, located in row 19730 by printing it out, including two adjacent rows:

In [97]:
metadataDF.iloc[19729,]

adult                                                                False
belongs_to_collection                                                  NaN
budget                                                                   0
genres                   [{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...
homepage                                                               NaN
id                                                                   82663
imdb_id                                                          tt0113002
original_language                                                       en
original_title                                                Midnight Man
overview                 British soldiers force a recently captured IRA...
popularity                                                             NaN
poster_path                                                            NaN
production_companies                                                   NaN
production_countries     

In [98]:
metadataDF.iloc[19730,]

adult                                                   - Written by Ørnås
belongs_to_collection                                             0.065736
budget                                    /ff9qCepilowshEtG2GYWwzt2bs4.jpg
genres                   [{'name': 'Carousel Productions', 'id': 11176}...
homepage                 [{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...
id                                                              1997-08-20
imdb_id                                                                  0
original_language                                                    104.0
original_title                    [{'iso_639_1': 'en', 'name': 'English'}]
overview                                                          Released
popularity                                                             NaN
poster_path                                                   Midnight Man
production_companies                                                 False
production_countries     

So, it looks like rows 19729 and 19730 are actually one entry that got split into two in the middle. You can see in the image below, that the description of Midnightman on IMDB was written by Ørnås:

![Image of midnightman](files/midnightMan.jpeg)

Upon checking the movies_metadata.csv file in notepad, there seems to be a new line character right before the words __- Written by Ørnås__, which causes the `Pandas` to assume it's a linebreak and split this entry right at the linebreak. I've fixed the issue by removing that linebreak. The same is done for the other two entries in rows 29503 and 35587. Let's reload the dataset now:

In [171]:
metadataDF = pd.read_csv("C:\\Users\\vosilr2\\Documents\\NAMV_MLcontest\\data\\movies_metadata.csv", low_memory=False)
print(metadataDF.adult.unique())

[False  True]


Now, let's convert the `budget` and `revenue` columns from string into numeric values:

In [173]:
metadataDF.budget = pd.to_numeric(metadataDF.budget)
metadataDF.revenue = pd.to_numeric(metadataDF.revenue)

In [174]:
metadataDF.budget.value_counts()

0            36573
5000000        286
10000000       259
20000000       243
2000000        242
15000000       226
3000000        223
25000000       206
1000000        197
30000000       190
4000000        181
6000000        174
12000000       172
40000000       167
8000000        155
500000         142
35000000       141
7000000        137
50000000       124
1500000        120
60000000       110
18000000       105
3500000        103
2500000        100
13000000        89
9000000         87
11000000        83
14000000        76
16000000        75
17000000        71
             ...  
10920000         1
786675           1
2515000          1
172000           1
747487           1
1880006          1
22361            1
1105736          1
280379           1
25485000         1
17080000         1
270000           1
48740            1
3849796          1
12156348         1
1220000          1
155000           1
171200           1
679000           1
8575000          1
5037000          1
4185000     

It looks like there is a huge portion of movies that has no budget data. Let's check how many of the movies actually have non-zero observations for budget and revenue:

In [178]:
print(f'There are {len(metadataDF[metadataDF.budget != 0])} movies with non-zero values for Budget.')
print(f'There are {len(metadataDF[metadataDF.revenue != 0])} movies with non-zero values for Revenue.')

There are 8890 movies with non-zero values for Budget.
There are 7408 movies with non-zero values for Revenue.


Let's see how many observations we have with data for, both, revenue and budget:

In [181]:
n = len(metadataDF[(metadataDF.revenue != 0) & (metadataDF.budget != 0)])
print(n)

5381


It doesn't look good:

In [191]:
print(f'Only {n} observations ({np.round(n/len(metadataDF)*100,1)}% of original data) can be used for the movie profitability prediction.')

Only 5381 observations (11.8% of original data) can be used for the movie profitability prediction.


In [194]:
metadataDF.head(3)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92


In [281]:
newDF = metadataDF[(metadataDF.revenue != 0) & (metadataDF.budget != 0)]
DF = metadataDF.join(other = df.set_index("id"), on="id", how="left")

In [283]:
DF2 = DF[(DF.revenue != 0) & (DF.budget != 0)]

In [286]:
print(DF.budget.sum())
print(DF2.budget.sum())
print(newDF.budget.sum())

print(DF.shape)
print(DF2.shape)
print(newDF.shape)
print(DF.columns)


192319916913
167552122038
167321097122
(45539, 26)
(5393, 26)
(5381, 24)
Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count', 'cast', 'crew'],
      dtype='object')


In [277]:
def tomHanks(dictionary, actorId):
    for k, v in dictionary.items():
        if k==actorId:
            return v#['cast_id']
        else:
            return np.NaN

tomHanksDF = df.cast.apply(lambda x: tomHanks(x, 31)).dropna()
len(tomHanksDF)

39

In [239]:
actorsList = []
def actorList(dictionary):
    for k in dictionary.keys():
        if k not in actorsList:
            actorsList.append(k) 

In [241]:
df.cast.apply(actorList)
print(actorsList)

KeyboardInterrupt: 

In [230]:
df.cast[0]

{31: {'cast_id': 14,
  'character': 'Woody (voice)',
  'credit_id': '52fe4284c3a36847f8024f95',
  'gender': 2,
  'id': 31,
  'name': 'Tom Hanks',
  'order': 0,
  'profile_path': '/pQFoyx7rp09CJTAb932F2g8Nlho.jpg'},
 7167: {'cast_id': 16,
  'character': 'Mr. Potato Head (voice)',
  'credit_id': '52fe4284c3a36847f8024f9d',
  'gender': 2,
  'id': 7167,
  'name': 'Don Rickles',
  'order': 2,
  'profile_path': '/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg'},
 7907: {'cast_id': 19,
  'character': 'Hamm (voice)',
  'credit_id': '52fe4284c3a36847f8024fa9',
  'gender': 2,
  'id': 7907,
  'name': 'John Ratzenberger',
  'order': 5,
  'profile_path': '/yGechiKWL6TJDfVE2KPSJYqdMsY.jpg'},
 8655: {'cast_id': 24,
  'character': 'Sergeant (voice)',
  'credit_id': '52fe4284c3a36847f8024fb9',
  'gender': 2,
  'id': 8655,
  'name': 'R. Lee Ermey',
  'order': 10,
  'profile_path': '/r8GBqFBjypLUP9VVqDqfZ7wYbSs.jpg'},
 8873: {'cast_id': 20,
  'character': 'Bo Peep (voice)',
  'credit_id': '52fe4284c3a36847f8024fad',
  

#### Keywords:

In [ ]:
keywordsDF = pd.read_csv("C:\\Users\\vosilr2\\Documents\\NAMV_MLcontest\\data\\keywords.csv")
keywordsDF.head()